# Data Analysis
This notebook is used to analyze the data collected from live runs. It includes various visualizations and statistical analyses to understand the performance of the runner

## TOC:

In [5]:
from utilis.helper import extract_global_json, extract_json
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from statsmodels.api import OLS, add_constant
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error

## Try running time series cross-validation with external regressors

In [11]:
def external_factor_importance(external_parameters: list, constant_parameters: list, target_columns: list):

    output_folder = extract_global_json('output_folder')
    # Prepare features including external factors
    all_features = []
    all_targets = []
    
    # loop through each dataset
    for folder_name in os.listdir(output_folder):
        csv_file_path = os.path.join(output_folder, folder_name, f"{folder_name}_streams.csv")
        json_file_path = os.path.join(output_folder, folder_name, f"{folder_name}_overall.json")
        # get the csv_file into a DataFrame
        df = pd.read_csv(csv_file_path)
        overall_data = extract_json(json_file_path)

        # remove rows with NaN values in the target column
        df = df.dropna()
        features = df[external_parameters].values
        targets = df[target_columns].values

        # Flatten weather dict if present
        flat = overall_data.copy()
        if "weather" in flat and isinstance(flat["weather"], dict):
            for k, v in flat["weather"].items():
                flat[f"{k}"] = v
            del flat["weather"]

        external_factors = np.tile(
            [flat[param] for param in constant_parameters],
            (len(df), 1)
        )
        # external_factors = np.full((len(df), 2), [overall_data['weather']['temp'], overall_data['weather']['humidity']])
        
        all_features.append(np.hstack([features, external_factors]))
        all_targets.append(targets)
    
    x = np.vstack(all_features)
    y = np.vstack(all_targets)
    # y = np.hstack(all_targets)
    
    # Feature names
    feature_names = external_parameters + constant_parameters

    # make them into a pandas DataFrame
    df_x = pd.DataFrame(x, columns=feature_names)
    df_y = pd.DataFrame(y, columns=target_columns)


    return df_x, df_y

df_x, df_y = external_factor_importance(
    external_parameters=["distance_m", "pace_efficiency", "smooth_heartrate_bps", "smooth_velocity_mps", "smooth_altitude_m", "diff_altitude_mps", "smooth_headwind_mps", "smooth_crosswind_mps", "smooth_grade_percent"],
    constant_parameters=["temp", "humidity", "pressure", "uvindex", "total_elevation_gain", "precip"],
    # target_column=["diff_heartrate_bpm"])
    # target_column=["diff_heartrate_shift_bpm"])
    # target_column=["acceleration_mps2"])
    # target_column=["acceleration_shift_mps2"])
    target_columns=["diff_heartrate_bps2", "diff_velocity_mps2"])


### Random Forest Regressor

In [12]:
# Create multi-output regressor
rf_multi = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
rf_multi.fit(df_x, df_y)

# Extract feature importance for each target
importance_results = {}

for i, target in enumerate(df_y.columns):
    importance_df = pd.DataFrame({
        'feature': df_x.columns,
        'importance': rf_multi.estimators_[i].feature_importances_
    }).sort_values('importance', ascending=False)
    
    importance_results[target] = importance_df
    
    print(f"\nFeature importance for {target}:")
    print(importance_df)


Feature importance for diff_heartrate_bps2:
                 feature  importance
0             distance_m    0.296224
3    smooth_velocity_mps    0.266462
2   smooth_heartrate_bps    0.203632
5      diff_altitude_mps    0.048730
1        pace_efficiency    0.035930
7   smooth_crosswind_mps    0.032620
8   smooth_grade_percent    0.031808
6    smooth_headwind_mps    0.030063
4      smooth_altitude_m    0.029803
10              humidity    0.007614
9                   temp    0.006856
12               uvindex    0.003688
11              pressure    0.003026
13  total_elevation_gain    0.002898
14                precip    0.000647

Feature importance for diff_velocity_mps2:
                 feature  importance
3    smooth_velocity_mps    0.343062
1        pace_efficiency    0.324694
0             distance_m    0.077132
2   smooth_heartrate_bps    0.063310
4      smooth_altitude_m    0.047399
6    smooth_headwind_mps    0.029749
7   smooth_crosswind_mps    0.026320
8   smooth_grade_percen

### Multivariate Regression

In [13]:
for i, target in enumerate(df_y.columns):
    print(f"{'='*10} OLS Regression Analysis for: {target} {'='*10} \n")
    
    # # Extract target variable
    # y_target = y[:, i]
    y = df_y[target]
    # Add constant for intercept
    df_x_const = add_constant(df_x)
    # feature_names_with_const = ['const'] + feature_names
    
    # Fit OLS model
    model = OLS(y, df_x_const).fit()

    # Print summary
    print(model.summary())

========== OLS Regression Analysis for: diff_heartrate_bps2 ========== 

                             OLS Regression Results                            
Dep. Variable:     diff_heartrate_bps2   R-squared:                       0.571
Model:                             OLS   Adj. R-squared:                  0.570
Method:                  Least Squares   F-statistic:                     585.8
Date:                 Fri, 08 Aug 2025   Prob (F-statistic):               0.00
Time:                         17:33:49   Log-Likelihood:                 25908.
No. Observations:                 6611   AIC:                        -5.178e+04
Df Residuals:                     6595   BIC:                        -5.168e+04
Df Model:                           15                                         
Covariance Type:             nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------

### Elastic Net Regression

In [15]:
# Elastic Net Regression
from sklearn.linear_model import ElasticNet

for i, target in enumerate(df_y.columns):
    print(f"{'='*10} Elastic Regression Analysis for: {target} {'='*10} \n")
    
    # # Extract target variable
    # y_target = y[:, i]
    y = df_y[target]

    # Define the model
    model = ElasticNet(alpha=0.1, l1_ratio=0.0, max_iter=10000)  # Adjust alpha and l1_ratio as needed
    # Fit the model
    model.fit(df_x, y)
    # predict pace efficiency based on the model
    y_pred = model.predict(df_x)

    print(model.score(df_x, y))
# # # Plot the data and the fitted line
# plt.figure(figsize=(10, 6))
# plt.plot(data["distance_m"], y, label='Data Points')
# # plt.scatter(data["distance_m"], y, label='Data Points')
# plt.plot(data["distance_m"], y_pred, color='red', label='Fitted Line')

========== Elastic Regression Analysis for: diff_heartrate_bps2 ========== 



c:\Users\katzr\Desktop\University\Projects\Marathon_Simulation\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.135e-01, tolerance: 3.562e-05
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


0.46008563609803765
========== Elastic Regression Analysis for: diff_velocity_mps2 ========== 

0.10860001674368136


c:\Users\katzr\Desktop\University\Projects\Marathon_Simulation\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.433e+01, tolerance: 7.409e-03
Linear regression models with a zero l1 penalization strength are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(


### Polynomial Regression

In [22]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import itertools

def polynomial_ols_regression(df_x, df_y, degree=2, interaction_only=False, include_bias=False):
    """
    Perform polynomial OLS regression for multiple targets
    """
    results = {}
    
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=include_bias)
    X_poly = poly.fit_transform(df_x)
    
    # Get feature names for polynomial features
    feature_names = poly.get_feature_names_out(df_x.columns)
    
    # Create DataFrame for polynomial features
    df_x_poly = pd.DataFrame(X_poly, columns=feature_names)
    
    print(f"Original features: {df_x.shape[1]}")
    print(f"Polynomial features (degree {degree}): {df_x_poly.shape[1]}")
    print(f"Interaction only: {interaction_only}")
    
    for target in df_y.columns:
        print(f"\n{'='*20} Polynomial OLS for: {target} {'='*20}")
        
        y = df_y[target]
        
        # Add constant for intercept
        df_x_poly_const = add_constant(df_x_poly)
        
        # Fit OLS model
        model = OLS(y, df_x_poly_const).fit()
        
        # Feature importance based on coefficients (excluding intercept)
        coefficients = model.params[1:]  # Exclude intercept
        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'coefficient': coefficients,
            'abs_coefficient': np.abs(coefficients),
            'p_value': model.pvalues[1:]  # Exclude intercept p-value
        }).sort_values('abs_coefficient', ascending=False)
        
        # Filter significant features
        significant_features = feature_importance[feature_importance['p_value'] < 0.05]
        
        results[target] = {
            'model': model,
            'poly_transformer': poly,
            'feature_importance': feature_importance,
            'significant_features': significant_features,
            'r_squared': model.rsquared,
            'adj_r_squared': model.rsquared_adj,
            'n_significant': len(significant_features)
        }
        
        print(f"R-squared: {model.rsquared:.4f}")
        print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")
        print(f"Number of significant features (p<0.05): {len(significant_features)}")
        
        print(f"\nTop 10 most important features by coefficient magnitude:")
        print(feature_importance.head(10)[['feature', 'coefficient', 'p_value']])
        
        if len(significant_features) > 0:
            print(f"\nTop 10 significant features (p<0.05):")
            print(significant_features.head(10)[['feature', 'coefficient', 'p_value']])
    
    return results

# Run polynomial regression with degree 2
poly_results_deg2 = polynomial_ols_regression(df_x, df_y, degree=2, interaction_only=False)

Original features: 15
Polynomial features (degree 2): 135
Interaction only: False

==================== Polynomial OLS for: diff_heartrate_bps2 ====================
R-squared: 0.7202
Adjusted R-squared: 0.7152
Number of significant features (p<0.05): 94

Top 10 most important features by coefficient magnitude:
                                                             feature  \
diff_altitude_mps                                  diff_altitude_mps   
pace_efficiency                                      pace_efficiency   
smooth_heartrate_bps                            smooth_heartrate_bps   
smooth_velocity_mps                              smooth_velocity_mps   
smooth_grade_percent                            smooth_grade_percent   
smooth_crosswind_mps                            smooth_crosswind_mps   
pace_efficiency precip                        pace_efficiency precip   
smooth_altitude_m                                  smooth_altitude_m   
diff_altitude_mps^2                     